In [1]:
import requests
import json
from datetime import datetime


In [2]:
API_BASE = "http://localhost:3000/api"  # adjust if your server runs elsewhere


In [3]:
def validate_license(key, signature, client=None):
    """
    Validate license from the server.
    Returns JSON response with `valid: True/False` and license info.
    """
    payload = {
        "key": key,
        "signature": signature
    }
    if client:
        payload["client"] = client

    resp = requests.post(f"{API_BASE}/license/validate", json=payload)
    try:
        return resp.json()
    except:
        return {"valid": False, "reason": "server_error", "status_code": resp.status_code}


def update_license_usage(key, client=None, meta=None):
    """
    Update license record (last used, device id, etc.)
    """
    payload = {}
    if client:
        payload["client"] = client
    if meta:
        payload["meta"] = meta

    resp = requests.post(f"{API_BASE}/license/{key}", json=payload)
    try:
        return resp.json()
    except:
        return {"success": False, "reason": "server_error", "status_code": resp.status_code}


In [10]:
import hashlib
import subprocess
import uuid
import os
import json
import requests
def get_system_client_id():
    # Try Windows UUID
    try:
        if os.name == "nt":
            output = subprocess.check_output(
                ["wmic", "csproduct", "get", "uuid"], text=True
            ).splitlines()
            uuid_val = [x.strip() for x in output if x.strip() and x.strip() != "UUID"]
            if uuid_val:
                return f"win-{uuid_val[0]}"
    except:
        pass

    # Try Linux machine-id
    try:
        if os.path.exists("/etc/machine-id"):
            mid = open("/etc/machine-id").read().strip()
            return f"linux-{mid}"
    except:
        pass

    # Try macOS IOPlatformUUID
    try:
        if sys.platform == "darwin":
            output = subprocess.check_output(
                ["ioreg", "-rd1", "-c", "IOPlatformExpertDevice"], text=True
            )
            for line in output.splitlines():
                if "IOPlatformUUID" in line:
                    uuid_val = line.split("=")[1].strip().strip('"')
                    return f"mac-{uuid_val}"
    except:
        pass

    # Fallback → stable hashed MAC
    mac = uuid.getnode()
    return "mac-hash-" + hashlib.sha256(str(mac).encode()).hexdigest()[:16]



In [11]:
def load_license_file(path="license.json"):
    if not os.path.exists(path):
        raise FileNotFoundError("license.json not found")

    with open(path, "r") as f:
        data = json.load(f)

    return data.get("key"), data.get("signature")


In [17]:
def validate_license(key, signature, client_id):
    url = "http://localhost:3000/api/license/validate"

    body = {
        "key": key,
        "signature": signature,
        "client": client_id
    }

    try:
        res = requests.post(url, json=body, timeout=5)
        return res.json()
    except Exception as e:
        return {"valid": False, "reason": str(e)}


In [ ]:
import json

def main():
    # 1. Read license.json
    LICENSE_KEY, LICENSE_SIG = load_license_file()

    # 2. Get system hardware-based stable ID
    CLIENT_ID = get_system_client_id()

    print("Using client ID:", CLIENT_ID)

    # 3. Validate with server
    result = validate_license(LICENSE_KEY, LICENSE_SIG, CLIENT_ID)

    print("Validation Response:")
    print(json.dumps(result, indent=2))

main()
    


Using client ID: xxxx
Validation Response:
{
  "valid": false,
  "reason": "client_mismatch"
}


In [ ]:
win-4C4C4544-0043-3210-8059-C4C04F5A4B33